# Geometric distribution example

In [1]:
# Load generated data from file:
import pickle

from evaluation_icml2022 import *

experiments = {}
num_chains = 10
configs = [
    (L, alpha, K) for L in [5, 2] for alpha in [1.0, 0.5, 0.1] for K in [0]
]
for L, alpha, K in configs:
    key = toconfigstr(L, alpha, K)
    experiments[key] = []
    for i in range(num_chains):
        filename = f"lookahead_samples/geometric_{i}__count1000_eps0.1_L{L}_alpha{alpha}_K{K}.pickle"
        with open(filename, "rb") as f:
            experiments[key].append(pickle.load(f))

In [2]:
values = {}
chains = {}
for config, runs in experiments.items():
    print(f"{config}")
    thinned_runs = thin_runs(runs)
    chains.update(collect_chains(thinned_runs, config=config))
    values.update(collect_values(thinned_runs, config=config))
    print_running_time(runs, thinned_runs)

L=5
npdhmc: 355.91s    0.0356s per sample (after thinning)
L=5, α=0.5
npdhmc-persistent: 362.65s    0.0363s per sample (after thinning)
L=5, α=0.1
npdhmc-persistent: 366.57s    0.0367s per sample (after thinning)
L=2
npdhmc: 152.71s    0.0153s per sample (after thinning)
L=2, α=0.5
npdhmc-persistent: 150.25s    0.0150s per sample (after thinning)
L=2, α=0.1
npdhmc-persistent: 149.60s    0.0150s per sample (after thinning)


In [3]:
def total_variational_distance(samples, p = 0.2):
    samples = sorted(samples)
    N = len(samples)
    mx = int(samples[-1])
    freq = [0 for _ in range(mx + 1)]
    for sample in samples:
        freq[int(sample)] += 1
    dist = 0
    for x in range(1,mx + 1):
        dist += abs(freq[x] / N - (1 - p)**(x-1)*p)
    dist += (1 - p)**mx # unsampled tail
    return dist / 2
import torch
print("Total variation distance from the ground truth (mean over all chains):")
for method in chains.keys():
    tvds = torch.tensor([total_variational_distance(chain) for chain in chains[method]])
    std, mean = torch.std_mean(tvds)
    print(f"{legend_str(method)}:  {mean:.4f} +- {std:.4f} (std)")

Total variation distance from the ground truth (mean over all chains):
NP-DHMC (L=5):  0.0524 +- 0.0069 (std)
NP-DHMC pers. (L=5, α=0.5):  0.0464 +- 0.0074 (std)
NP-DHMC pers. (L=5, α=0.1):  0.0461 +- 0.0083 (std)
NP-DHMC (L=2):  0.0768 +- 0.0181 (std)
NP-DHMC pers. (L=2, α=0.5):  0.0570 +- 0.0115 (std)
NP-DHMC pers. (L=2, α=0.1):  0.0534 +- 0.0058 (std)
